In [1]:
import os
import sys
sys.path.append('../')

from library import Graph, GraphLib

In [2]:
graphLib = GraphLib(config='./config_test.yaml')
print(graphLib)

Graph Library with configurations:
{'dataset': 'CIFAR10', 'classes': 10, 'input_channels': 3, 'image_size': 64, 'data_dir': '/scratch/gpfs/stuli/pytorch_datasets/', 'manual_dir': None, 'models_dir': '/scratch/gpfs/stuli/cnnbench_models/', 'epochs': 10, 'train_batch_size': 256, 'test_batch_size': 512, 'optimizer': 'Adam', 'lr': 0.001, 'scheduler': 'CosineAnnealingWarmRestarts', 'scheduler_args': {'T_0': 5}, 'module_vertices': 4, 'head_vertices': 4, 'max_edges': 9, 'max_modules': 2, 'base_ops': ['conv5x5-bn-relu', 'conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3'], 'default_channels': 128, 'default_stride': 1, 'flatten_ops': ['global-avg-pool', 'flatten'], 'dense_ops': ['dense-1024', 'dense-4096', 'dropout-p5', 'dropout-p4'], 'hash_algo': 'sha256'}
Number of graphs: 0


In [3]:
graphLib.build_library()

Generating modules...
Using 4 vertices, 4 labels, max 9 edges
	Generated up to 3 vertices: 8 modules (8 without hashing)
	Generated up to 4 vertices: 156 modules (168 without hashing)

Generating heads...
Using 4 vertices, 6 labels
	Generated up to 4 vertices: 2 heads (2 without hashing)

Generating graphs...
Using max 2 modules



	Generated up to 1 modules: 312 graphs



	Generated up to 2 modules: 48824 graphs
Graph library created! 
48824 graphs within the design space.


In [4]:
graphLib.save_dataset('./dataset.json')

Dataset saved to: ./dataset.json


In [10]:
graphLib_new = GraphLib.load_from_dataset('./dataset.json')
graphLib_new

Graph Library with configurations:
{'dataset': 'CIFAR10', 'classes': 10, 'input_channels': 3, 'image_size': 28, 'data_dir': '/scratch/gpfs/stuli/pytorch_datasets/', 'manual_dir': None, 'train_batch_size': 256, 'test_batch_size': 512, 'module_vertices': 4, 'head_vertices': 4, 'max_edges': 9, 'max_modules': 2, 'base_ops': ['conv5x5-bn-relu', 'conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3'], 'default_channels': 128, 'default_stride': 1, 'flatten_ops': ['global-avg-pool', 'flatten'], 'dense_ops': ['dense-1024', 'dense-4096', 'dropout-p5', 'dropout-p4'], 'hash_algo': 'sha256'}
Number of graphs: 48824

In [3]:
import sys
sys.path.append('../')

from library import Graph, GraphLib

graphLib = GraphLib.load_from_dataset('./dataset.json')

graphLib.library[1]

Model parameters: None
Accuracies: {'train': None, 'val': None, 'test': None}
Embedding: None
Hash: 68510bcf50e51d05fef802b1af7a08334b76275fae640836ceb787e5af9dcd65
Module:
[[0 1 0]
 [0 0 1]
 [0 0 0]]
Labels:['input', 'conv5x5-bn-relu', 'output']
Module:
[[0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]]
Labels:['input', 'flatten', 'dense_classes', 'output']

In [6]:
import sys
sys.path.append('../')

from library import Graph, GraphLib

from model_builder import CNNBenchModel

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

graphLib = GraphLib.load_from_dataset('./dataset.json')

inp = torch.rand(64, 3, graphLib.config['image_size'], graphLib.config['image_size'])
inp = inp.to(device)

# for i in range(len(graphLib)):
#     model = CNNBenchModel(graphLib.config, graphLib.library[i])
#     model.to(device)
#     logits = model(inp)
#     print(f'\r{i}/{len(graphLib)}', end='')
model = CNNBenchModel(graphLib.config, graphLib.library[1])
print(model)

CNNBenchModel(
  (op_m0_v1): Sequential(
    (0): Conv2d(3, 128, kernel_size=(5, 5), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (proj_m0_v1): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
  (proj_m0): Conv2d(3, 128, kernel_size=(1, 1), stride=(1, 1))
  (op_m1_v2): Linear(in_features=460800, out_features=10, bias=True)
)


In [7]:
model.get_params()

4618390

In [2]:
from model_trainer import worker

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Training model:')
print(graphLib.library[1])
print()

worker(graphLib.config, graphLib.library[1], device)

Training model:
Model parameters: None
Accuracies: {'train': None, 'val': None, 'test': None}
Embedding: None
Hash: 68510bcf50e51d05fef802b1af7a08334b76275fae640836ceb787e5af9dcd65
Module:
[[0 1 0]
 [0 0 1]
 [0 0 0]]
Labels:['input', 'conv5x5-bn-relu', 'output']
Module:
[[0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]]
Labels:['input', 'flatten', 'dense_classes', 'output']


Loading Training dataset
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /scratch/gpfs/stuli/pytorch_datasets/
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=32, interpolation=bilinear)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
           )
Loading Test dataset
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: /scratch/gpfs/stuli/pytorch_datasets/
    Split: Test
    StandardTransform
Transform: Compose(
               Resize(size=32, int

Train Epoch: 4 [ 15360/ 50000 (31%)]	Learning Rate: 0.000047	Loss: 0.739877
Train Epoch: 4 [ 17920/ 50000 (36%)]	Learning Rate: 0.000040	Loss: 0.793790
Train Epoch: 4 [ 20480/ 50000 (41%)]	Learning Rate: 0.000034	Loss: 0.887421
Train Epoch: 4 [ 23040/ 50000 (46%)]	Learning Rate: 0.000029	Loss: 0.815059
Train Epoch: 4 [ 25600/ 50000 (51%)]	Learning Rate: 0.000023	Loss: 0.761585
Train Epoch: 4 [ 28160/ 50000 (56%)]	Learning Rate: 0.000019	Loss: 0.855973
Train Epoch: 4 [ 30720/ 50000 (61%)]	Learning Rate: 0.000015	Loss: 0.795239
Train Epoch: 4 [ 33280/ 50000 (66%)]	Learning Rate: 0.000011	Loss: 0.825276
Train Epoch: 4 [ 35840/ 50000 (71%)]	Learning Rate: 0.000008	Loss: 0.764152
Train Epoch: 4 [ 38400/ 50000 (77%)]	Learning Rate: 0.000005	Loss: 0.821767
Train Epoch: 4 [ 40960/ 50000 (82%)]	Learning Rate: 0.000003	Loss: 0.734719
Train Epoch: 4 [ 43520/ 50000 (87%)]	Learning Rate: 0.000002	Loss: 0.841771
Train Epoch: 4 [ 46080/ 50000 (92%)]	Learning Rate: 0.000001	Loss: 0.769467
Train Epoch:

Train Epoch: 9 [ 15360/ 50000 (31%)]	Learning Rate: 0.000047	Loss: 0.651075
Train Epoch: 9 [ 17920/ 50000 (36%)]	Learning Rate: 0.000040	Loss: 0.599322
Train Epoch: 9 [ 20480/ 50000 (41%)]	Learning Rate: 0.000034	Loss: 0.673050
Train Epoch: 9 [ 23040/ 50000 (46%)]	Learning Rate: 0.000029	Loss: 0.561582
Train Epoch: 9 [ 25600/ 50000 (51%)]	Learning Rate: 0.000023	Loss: 0.579881
Train Epoch: 9 [ 28160/ 50000 (56%)]	Learning Rate: 0.000019	Loss: 0.581424
Train Epoch: 9 [ 30720/ 50000 (61%)]	Learning Rate: 0.000015	Loss: 0.782312
Train Epoch: 9 [ 33280/ 50000 (66%)]	Learning Rate: 0.000011	Loss: 0.591047
Train Epoch: 9 [ 35840/ 50000 (71%)]	Learning Rate: 0.000008	Loss: 0.462567
Train Epoch: 9 [ 38400/ 50000 (77%)]	Learning Rate: 0.000005	Loss: 0.575339
Train Epoch: 9 [ 40960/ 50000 (82%)]	Learning Rate: 0.000003	Loss: 0.549632
Train Epoch: 9 [ 43520/ 50000 (87%)]	Learning Rate: 0.000002	Loss: 0.566401
Train Epoch: 9 [ 46080/ 50000 (92%)]	Learning Rate: 0.000001	Loss: 0.502077
Train Epoch:

In [8]:
import manual_models

for model_name in manual_models.SUPPORTED_MODELS:
    print(f'Tesing model: {model_name}')
    model_graph = manual_models.get_manual_graph(graphLib.config, model_name)
    cnn_model = CNNBenchModel(graphLib.config, model_graph)
    print(f'\tModel params: {cnn_model.get_params()}')

Tesing model: lenet
	Model params: 11519784
Tesing model: alexnet
	Model params: 5931478
Tesing model: vgg11
	Model params: 15078934
Tesing model: vgg13
	Model params: 15272214
Tesing model: vgg19
	Model params: 26748694
Tesing model: resnet18
	Model params: 12500502
Tesing model: resnet34
	Model params: 24297686
Tesing model: resnet50
	Model params: 62694422
Tesing model: resnet101
	Model params: 126294550
Tesing model: resnet152
	Model params: 178676758
Tesing model: shufflenet
	Model params: 40371598
Tesing model: mobilenet
	Model params: 11953622
Tesing model: googlenet
	Model params: 25794326
